In [1]:
import pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatCohere
from langchain.schema.output_parser import StrOutputParser

from dotenv import load_dotenv
import os

/home/codespace/.python/current/lib/python3.10/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
load_dotenv()
COHERE_API_KEY = os.getenv('COHERE_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENV = os.getenv("PINECONE_ENV")

In [4]:
embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_ENV,  # next to api key in console
)

index_name = "ml-interview-warmup"
embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

index = pinecone.Index(index_name)
vectorstore = Pinecone(index, embedding_function, "text")
retriever = vectorstore.as_retriever()

In [5]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator

# Define your desired data structure.
class InterviewQnA(BaseModel):
    question: str = Field(description="question about a topic")
    answer: str = Field(description="answer for the question")

    # You can add custom validation logic easily with Pydantic.
    @validator('question')
    def question_ends_with_question_mark(cls, field):
        if field[-1] != '?':
            raise ValueError("Badly formed question!")
        return field

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=InterviewQnA)

template = """Topic: {user_topic}
Context: {context}

Based on the context above, generate a question and answer pair about the topic.
{format_instructions}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["user_topic", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)


model = ChatCohere()

chain = prompt | model | StrOutputParser()

user_topic = 'prompting language model'
context = retriever.invoke(user_topic)

output = chain.invoke({"user_topic": user_topic, "context": context})
qna_pair = parser.invoke(output)
qna_pair

InterviewQnA(question='What is prompt engineering?', answer='Prompt engineering, also known as In-Context Prompting, refers to methods for how to communicate with LLM to steer its behavior for desired outcomes without updating the model weights. The effect of prompt engineering methods can vary among models, thus requiring heavy experimentation and heuristics.')

In [6]:
user_answer = "I think prompt engineering is a method for communicating with LLM to steer its behavior for desired outcomes without updating the model weights. It is an empirical science and the effect of prompt engineering methods can vary a lot among models, thus requiring heavy experimentation and heuristics."

template = """Topic: {user_topic}
Context: {context}

Question: {question}
Examiner's answer: {examiner_answer}
User's answer: {user_answer}

Act as an examiner and objectively rate the user's answer on a scale of 1-5 based on:
- How accurately it answers the question based on the given context
- How fully it covers the key points in the examiner's answer
- How well structured, clear and concise the response is

Based on the question, context, examiner's answer and user's answer above, objectively provide a score between 1-5 and a short justification explaining your rating:
{format_instructions}
"""

# Define your desired data structure.
class AnswerReview(BaseModel):
    score: int = Field(description="score for user's answer")
    justification: str = Field(description="justification explaining the score")

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=AnswerReview)

prompt = PromptTemplate(
    template=template,
    input_variables=["user_topic", "context", "question", "examiner_answer", "user_answer"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = prompt | model | StrOutputParser()

output = chain.invoke({"user_topic": user_topic, "context": context, "question": qna_pair.question, "examiner_answer": qna_pair.answer, "user_answer": user_answer})
answer_review = parser.invoke(output)
answer_review

AnswerReview(score=4, justification="User's answer is a direct paraphrase of the examiner's answer, and covers the key points. It is clear and well-structured.")